# Day 03 — Tool routing (structured outputs)

Tool routing means **choosing a tool** based on the task. We ask the model to output a JSON decision.

We will cover:
- Defining tools
- Asking for a structured routing decision
- Executing the chosen tool


## 1) Define tools
We’ll define three tools: summarize, plan, and estimate.


In [ ]:
import json
from openai import OpenAI

client = OpenAI()
MODEL = "gpt-4o-mini"

tools = {
    "summarize": "Summarize a passage in 2 sentences.",
    "plan": "Create a step-by-step plan.",
    "estimate": "Provide a rough time estimate in hours.",
}


## 2) Ask for a routing decision
We request JSON with the chosen tool and arguments.


In [ ]:
task = "Read this text and summarize it: Artificial intelligence is a field..."

router_prompt = f"""
You are a router. Choose the best tool for the task.
Available tools: {list(tools.keys())}
Return JSON: {{"tool": "...", "arguments": "..."}}

Task: {task}
"""

response = client.responses.create(model=MODEL, input=router_prompt, temperature=0.1)
router_output = response.output_text
router_output


## 3) Parse and execute
We parse the JSON and run the correct tool handler.


In [ ]:
decision = json.loads(router_output)

if decision["tool"] == "summarize":
    tool_prompt = f"Summarize in 2 sentences: {decision['arguments']}"
elif decision["tool"] == "plan":
    tool_prompt = f"Create a 5-step plan: {decision['arguments']}"
else:
    tool_prompt = f"Estimate hours required: {decision['arguments']}"

result = client.responses.create(model=MODEL, input=tool_prompt, temperature=0.2).output_text
result


## 4) What to do next
Next we’ll add memory and state to keep long-running conversations coherent.
